In [1]:
import copy
import yaml
import pathlib
import numpy as np
import open3d as o3d
from dangin import Stitcher, dataloader_dmap, dataloader_ply

## Set initial config to work with

In [2]:
# Select one dataset
dataset = 'LA200'
root_path = str(pathlib.Path('').parent.resolve().parent)
config_path = root_path + "/" + "config.yaml"
dataset_path = root_path + f'/datasets/{dataset}'
with open(config_path, "rt") as f:
    config = yaml.safe_load(f.read())

## The following two cells will guide you to work with a rolling window approach.
### This means that instead of adding a new PCD (point cloud) and comparing it to ALL accumulated points it will only compare it to the rolling window points.

In [3]:
pcds_rolling_window = 5             # Represents how many previous point clouds are going to be used 
                                    # as matching reference for the new point cloud .
pcd_size = 128*128                  # Each point cloud is composed of 128*128 points, its self explanatory.

# When you sum point clouds Open3d appends the points at the end,
# rolling_window_total_points will indicate how many point we need
# to grab starting from the end such that we ensure to be taking 
# the N FULL point clouds define by rolling window.
rolling_window_total_points = pcds_rolling_window * pcd_size

vis_heigth, vis_width = 1000, 1000  # Window size for displaying pcd

### This equation shows the relationship of individual transforms between a PCD and their previous PCD. Therefore is <b>ESSENTIAL<b> to initialize the <b>FIRST PCD<b> w.r.t to world frame.  
$$
   T^{w}_{pcd_i} = T^{w}_{pcd_0} \cdot T^{pcd_0}_{pcd_1} \cdot T^{pcd_1}_{pcd_2} \cdot ... \cdot T^{pcd_{i-1}}_{pcd_i}
$$


### A couple of assumptions:
- We are assuming the the trayectory is continous and there are <b>NO<b> jumps in space, this means that the <b>NEXT<b> PCD should be located pretty close to the current PCD.
- We currently dont have any odometry data, therefore the best initialization pose for the new PCD is the current pose (transform) of the PCD.
   - In other words this means every new PCD will appear on top of the previous PCD (in reality is not the case because we \
   dont know how open3d defines the origin frame for each PCD, but assuming is the centroid of mass then yes to some level).

**<b>Comment:<b> ICP output is the best guess transformation from <b>CURRENT PCD<b> to <b>PREVIOUS PCD<b>:
 $$
   ICP_{output} = T^{pcd_{i-1}}_{pcd_i}
 $$

 

In [8]:
#  *** **************** ******************** ******************** ******************** ******************** ***
#  ******************** This approach is for stitching NEW point cloud with PREVIOUS. ******************** 
#  *** **************** ******************** ******************** ******************** ******************** ***
stitcher = Stitcher(**config["stitcher_params"])   
dataset = dataloader_dmap(dataset_path, ground_truth=True)

pcd_0 = o3d.geometry.PointCloud()
pcd_1 = o3d.geometry.PointCloud()

dmap_0, img_0 = next(dataset)

stitcher.generate_pointcloud(img_0, dmap_0, pcd_0)
pcd_0 = stitcher.outlier_removal(pcd_0)                                                   # First some preprocessing/outlier removal

pcd_history = pcd_history_down = copy.deepcopy(pcd_0)
history_points_xyz = np.array(pcd_history.points) 

vis = o3d.visualization.Visualizer()                                                      # *********************
vis.create_window(height=vis_heigth, width=vis_width)                                     # Setting up visualizer
vis.add_geometry(pcd_history_down)                                                        # *********************

for i, package in enumerate(dataset):
    dmap_1, img_1 = package
    stitcher.generate_pointcloud(img_1, dmap_1, pcd_1)
    pcd_1 = stitcher.outlier_removal(pcd_1)                                              # First some preprocessing/outlier removal
    
    pcd_1_temp = copy.deepcopy(pcd_1)
    
    if len(history_points_xyz) >= (rolling_window_total_points):                          # This if-statement makes sure that we have enough pcds to make a rolling window
        pass
    else:
        stitcher.point_cloud_init_pose(pcd_0, stitcher.current_transform)                 # The initial current_transform is w.r.t. world frame.   
        
    stitcher.point_cloud_init_pose(pcd_1, stitcher.current_transform)                     # As the name implies first initialize PCD on specified loc.
    result_icp = stitcher.global_and_icp_registration_v2(pcd_1, pcd_0)                    # Calculate the best transform starting between PCDs. This transformation is from current pcd to previous pcd
    current_transform = stitcher.current_transform @ result_icp.transformation            # Calculate the new current transform of the trayectory. 
    stitcher.set_current_transform(current_transform)                                     # This is useful for the next CONSECUTIVE PCD. As the starting loc.
    
    stitcher.transforms_history.append(result_icp.transformation)                         # This is useful to have a graph of all the transforms for further analysis                        
    
    pcd_history += pcd_1.transform(result_icp.transformation)                             # This represents the WHOLE PCD as we progress in time  
    
    vis.update_geometry(pcd_history_down)
    vis.poll_events()
    vis.update_renderer()
    
    history_points_xyz = np.array(pcd_history.points)                            
    history_points_rgb = np.array(pcd_history.colors) 
        
    if len(history_points_xyz) >= (rolling_window_total_points):                          # If we have enough points for rolling windows we proceed with it  
        pcd_0 = o3d.geometry.PointCloud()                                                 # it means that pcd_0 will not be just the old point cloud but a group of prev pcds.
        pcd_0.points = o3d.utility.Vector3dVector(history_points_xyz[-(rolling_window_total_points):, :]) # Get points information representing rolling window
        pcd_0.colors = o3d.utility.Vector3dVector(history_points_rgb[-(rolling_window_total_points):, :]) # Get points information representing rolling window
    else:
        pcd_0 = copy.deepcopy(pcd_1_temp)

    #print(f"Image: {i+1}")
    if (i+1) == 199:
        break
        
vis.destroy_window()
o3d.visualization.draw_geometries([pcd_history])

Image: 1
Image: 2
Image: 3
Image: 4
Image: 5
Image: 6
Image: 7
Image: 8
Image: 9
Image: 10


### If you want to use the WHOLE history of point clouds to do the allignment instead of rolling window just remove the if statments. (Compare both code examples to see the differences)

### Be AWARE that as you increase the history of point clouds the allignment process will take longer as the global allignment is evaluating ALL existing points.

In [5]:
#  *** **************** ******************** ******************** ******************** ******************** ***
#  ******************** This approach is for stitching NEW point cloud with PREVIOUS. ******************** 
#  *** **************** ******************** ******************** ******************** ******************** ***
stitcher = Stitcher(**config["stitcher_params"])   
dataset = dataloader_dmap(dataset_path, ground_truth=True)

pcd_0 = o3d.geometry.PointCloud()
pcd_1 = o3d.geometry.PointCloud()

dmap_0, img_0 = next(dataset)

stitcher.generate_pointcloud(img_0, dmap_0, pcd_0)
pcd_0 = stitcher.outlier_removal(pcd_0)                                                   # First some preprocessing/outlier removal
stitcher.point_cloud_init_pose(pcd_0, stitcher.current_transform)                         # The initial current_transform is w.r.t. world frame.   

pcd_history = pcd_history_down = copy.deepcopy(pcd_0)

vis = o3d.visualization.Visualizer()                                                      # *********************
vis.create_window(height=vis_heigth, width=vis_width)                                     # Setting up visualizer
vis.add_geometry(pcd_history_down)                                                        # *********************

for i, package in enumerate(dataset):
    dmap_1, img_1 = package
    stitcher.generate_pointcloud(img_1, dmap_1, pcd_1)
    pcd_1 = stitcher.outlier_removal(pcd_1)                                              # First some preprocessing/outlier removal
    
    pcd_1_temp = copy.deepcopy(pcd_1)           
        
    stitcher.point_cloud_init_pose(pcd_1, stitcher.current_transform)                     # As the name implies first initialize PCD on specified loc.
    result_icp = stitcher.global_and_icp_registration_v2(pcd_1, pcd_0)                    # Calculate the best transform starting between PCDs. This transformation is from current pcd to previous pcd
    current_transform = stitcher.current_transform @ result_icp.transformation            # Calculate the new current transform of the trayectory. 
    stitcher.set_current_transform(current_transform)                                     # This is useful for the next CONSECUTIVE PCD. As the starting loc.
    
    stitcher.transforms_history.append(result_icp.transformation)                         # This is useful to have a graph of all the transforms for further analysis. 
    
    pcd_history += pcd_1.transform(result_icp.transformation)                             # This represents the WHOLE PCD as we progress in time   
    
    vis.update_geometry(pcd_history_down)
    vis.poll_events()
    vis.update_renderer()
    
    pcd_0 = pcd_history

    #print(f"Image: {i+1}")
    if (i+1) == 199:
        break
        
vis.destroy_window()
o3d.visualization.draw_geometries([pcd_history])